In [1]:
# Goal of this model is to predict the Infant Survival
# Logistic Regression and RandomForest is used to predict the survival of Infant during birth. 
# Data can be downloaded from here: http://www.tomdrabas.com/data/LearningPySpark/births_train.csv.gz.
# Original schema of the Input file avaialble @ ftp://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/D

# Steps in Data Analysis
# 1. Highly correlated features are removed;
# 2. Run chi-square test for categorical columns
# 3. Fit the model with Logistic Regression and RandomForest

#Which parts this example conver ? delete not applicables
# 2. Logistic Regression
# 9. Bagging and Random Forest
 

In [2]:
# Goal is to predict the chance of Infant survival during birth

import pyspark.sql.types as typ

labels = [
    ('INFANT_ALIVE_AT_REPORT', typ.StringType()),
    ('BIRTH_YEAR', typ.IntegerType()),
    ('BIRTH_MONTH', typ.IntegerType()),
    ('BIRTH_PLACE', typ.StringType()),
    ('MOTHER_AGE_YEARS', typ.IntegerType()),
    ('MOTHER_RACE_6CODE', typ.StringType()),
    ('MOTHER_EDUCATION', typ.StringType()),
    ('FATHER_COMBINED_AGE', typ.IntegerType()),
    ('FATHER_EDUCATION', typ.StringType()),
    ('MONTH_PRECARE_RECODE', typ.StringType()),
    ('CIG_BEFORE', typ.IntegerType()),
    ('CIG_1_TRI', typ.IntegerType()),
    ('CIG_2_TRI', typ.IntegerType()),
    ('CIG_3_TRI', typ.IntegerType()),
    ('MOTHER_HEIGHT_IN', typ.IntegerType()),
    ('MOTHER_BMI_RECODE', typ.IntegerType()),
    ('MOTHER_PRE_WEIGHT', typ.IntegerType()),
    ('MOTHER_DELIVERY_WEIGHT', typ.IntegerType()),
    ('MOTHER_WEIGHT_GAIN', typ.IntegerType()),
    ('DIABETES_PRE', typ.StringType()),
    ('DIABETES_GEST', typ.StringType()),
    ('HYP_TENS_PRE', typ.StringType()),
    ('HYP_TENS_GEST', typ.StringType()),
    ('PREV_BIRTH_PRETERM', typ.StringType()),
    ('NO_RISK', typ.StringType()),
    ('NO_INFECTIONS_REPORTED', typ.StringType()),
    ('LABOR_IND', typ.StringType()),
    ('LABOR_AUGM', typ.StringType()),
    ('STEROIDS', typ.StringType()),
    ('ANTIBIOTICS', typ.StringType()),
    ('ANESTHESIA', typ.StringType()),
    ('DELIV_METHOD_RECODE_COMB', typ.StringType()),
    ('ATTENDANT_BIRTH', typ.StringType()),
    ('APGAR_5', typ.IntegerType()),
    ('APGAR_5_RECODE', typ.StringType()),
    ('APGAR_10', typ.IntegerType()),
    ('APGAR_10_RECODE', typ.StringType()),
    ('INFANT_SEX', typ.StringType()),
    ('OBSTETRIC_GESTATION_WEEKS', typ.IntegerType()),
    ('INFANT_WEIGHT_GRAMS', typ.IntegerType()),
    ('INFANT_ASSIST_VENTI', typ.StringType()),
    ('INFANT_ASSIST_VENTI_6HRS', typ.StringType()),
    ('INFANT_NICU_ADMISSION', typ.StringType()),
    ('INFANT_SURFACANT', typ.StringType()),
    ('INFANT_ANTIBIOTICS', typ.StringType()),
    ('INFANT_SEIZURES', typ.StringType()),
    ('INFANT_NO_ABNORMALITIES', typ.StringType()),
    ('INFANT_ANCEPHALY', typ.StringType()),
    ('INFANT_MENINGOMYELOCELE', typ.StringType()),
    ('INFANT_LIMB_REDUCTION', typ.StringType()),
    ('INFANT_DOWN_SYNDROME', typ.StringType()),
    ('INFANT_SUSPECTED_CHROMOSOMAL_DISORDER', typ.StringType()),
    ('INFANT_NO_CONGENITAL_ANOMALIES_CHECKED', typ.StringType()),
    ('INFANT_BREASTFED', typ.StringType())
]

schema = typ.StructType([
        typ.StructField(e[0], e[1], False) for e in labels
    ])

In [4]:
# create the dataframe using csv data
births = spark.read.csv('/FileStore/tables/births_train_csv-dc6be.gz', 
                        header=True, 
                        schema=schema)

In [5]:
# Specify our recode dictionary.

recode_dictionary = {
    'YNU': {
        'Y': 1,
        'N': 0,
        'U': 0
    }
}

In [6]:
# Goal is to predict whether the 'INFANT_ALIVE_AT_REPORT' is either 1 or 0. Hence dropping other features relate to infant and selecting important features.
selected_features = [
    'INFANT_ALIVE_AT_REPORT', 
    'BIRTH_PLACE', 
    'MOTHER_AGE_YEARS', 
    'FATHER_COMBINED_AGE', 
    'CIG_BEFORE', 
    'CIG_1_TRI', 
    'CIG_2_TRI', 
    'CIG_3_TRI', 
    'MOTHER_HEIGHT_IN', 
    'MOTHER_PRE_WEIGHT', 
    'MOTHER_DELIVERY_WEIGHT', 
    'MOTHER_WEIGHT_GAIN', 
    'DIABETES_PRE', 
    'DIABETES_GEST', 
    'HYP_TENS_PRE', 
    'HYP_TENS_GEST', 
    'PREV_BIRTH_PRETERM'
]

births_trimmed = births.select(selected_features)

In [7]:
# Specify the recoding methods.
import pyspark.sql.functions as func

def recode(col, key):        
    return recode_dictionary[key][col] 

def correct_cig(feat):
    return func \
        .when(func.col(feat) != 99, func.col(feat))\
        .otherwise(0)

rec_integer = func.udf(recode, typ.IntegerType())
births_trimmed.show()

+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+
INFANT_ALIVE_AT_REPORT|BIRTH_PLACE|MOTHER_AGE_YEARS|FATHER_COMBINED_AGE|CIG_BEFORE|CIG_1_TRI|CIG_2_TRI|CIG_3_TRI|MOTHER_HEIGHT_IN|MOTHER_PRE_WEIGHT|MOTHER_DELIVERY_WEIGHT|MOTHER_WEIGHT_GAIN|DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|
+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+
 N| 1| 29| 99| 99| 99| 99| 99| 99| 999| 999| 99| N| N| N| N| N|
 N| 1| 22| 29| 0| 0| 0| 0| 65| 180| 198| 18| N| N| N| N| N|
 N| 1| 38| 40| 0| 0| 0| 0| 63| 155| 167| 12| N| N| N| N| N|
 N| 1| 39| 42| 0| 0| 0| 0| 60| 128| 152| 24| N| N| N| N| Y|
 N| 1| 18| 99| 6| 4| 2| 2| 61| 110| 130| 20| N| N| N| N| N|
 N| 1| 32| 37| 0| 0| 0| 0| 66| 150| 162| 12| N| N| N| N| N|
 N| 1| 22| 25| 0| 0| 0| 0| 68| 155| 191| 36| N| N| N| N| N|
 N| 1| 25| 26| 0| 0| 0| 0| 64| 136| 169| 33| N| N| N| N| N|
 N| 1| 26| 32| 0| 0| 0| 0| 64| 140| 147| 7| N| N| N| N| N|
 N| 1| 39| 66| 0| 0| 0| 0| 65| 140| 150| 10| N| N| N| N| N|
 N| 1| 25| 22| 0| 0| 0| 0| 62| 145| 152| 7| N| N| N| N| N|
 N| 1| 33| 99| 0| 0| 0| 0| 65| 145| 145| 0| N| N| N| N| N|
 N| 1| 25| 99| 0| 0| 0| 0| 64| 140| 165| 25| U| U| U| U| U|
 N| 1| 29| 99| 0| 0| 0| 0| 60| 115| 120| 5| N| N| N| N| N|
 N| 1| 28| 29| 0| 0| 0| 0| 66| 320| 318| 0| N| N| Y| N| Y|
 N| 1| 23| 28| 0| 0| 0| 0| 64| 120| 141| 21| N| N| N| N| N|
 N| 1| 31| 41| 99| 99| 99| 99| 59| 106| 142| 36| N| N| N| N| Y|
 N| 1| 27| 99| 0| 0| 0| 0| 66| 213| 200| 0| N| N| N| N| N|
 N| 1| 28| 27| 0| 0| 0| 0| 66| 165| 224| 59| N| N| N| N| Y|
 N| 1| 34| 31| 0| 0| 0| 0| 70| 130| 134| 4| N| N| N| N| N|
+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+
only showing top 20 rows

In [8]:
# Correct the features related to the number of smoked cigarettes.
births_transformed = births_trimmed \
    .withColumn('CIG_BEFORE', correct_cig('CIG_BEFORE'))\
    .withColumn('CIG_1_TRI', correct_cig('CIG_1_TRI'))\
    .withColumn('CIG_2_TRI', correct_cig('CIG_2_TRI'))\
    .withColumn('CIG_3_TRI', correct_cig('CIG_3_TRI'))

In [9]:
# find out y/N/U columns
cols = [(col.name, col.dataType) for col in births_trimmed.schema]

YNU_cols = []

for i, s in enumerate(cols):
    if s[1] == typ.StringType():
        dis = births.select(s[0]) \
            .distinct() \
            .rdd \
            .map(lambda row: row[0]) \
            .collect()

        if 'Y' in dis:
            YNU_cols.append(s[0])

In [10]:
# DataFrames can transform the features in bulk while selecting features.

births.select([
        'INFANT_NICU_ADMISSION', 
        rec_integer(
            'INFANT_NICU_ADMISSION', func.lit('YNU')
        ) \
        .alias('INFANT_NICU_ADMISSION_RECODE')]
     ).take(5)

Out[ 9 ]: 
[Row(INFANT_NICU_ADMISSION=u'Y', INFANT_NICU_ADMISSION_RECODE=1),
 Row(INFANT_NICU_ADMISSION=u'Y', INFANT_NICU_ADMISSION_RECODE=1),
 Row(INFANT_NICU_ADMISSION=u'U', INFANT_NICU_ADMISSION_RECODE=0),
 Row(INFANT_NICU_ADMISSION=u'N', INFANT_NICU_ADMISSION_RECODE=0),
 Row(INFANT_NICU_ADMISSION=u'U', INFANT_NICU_ADMISSION_RECODE=0)]

In [11]:
# Transform all the YNU_cols in one using a list of transformations.
exprs_YNU = [
    rec_integer(x, func.lit('YNU')).alias(x) 
    if x in YNU_cols 
    else x 
    for x in births_transformed.columns
]

births_transformed = births_transformed.select(exprs_YNU)

In [12]:
# checking if the transformation is correct
births_transformed.select(YNU_cols[-5:]).show(5)

+------------+-------------+------------+-------------+------------------+
DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|
+------------+-------------+------------+-------------+------------------+
 0| 0| 0| 0| 0|
 0| 0| 0| 0| 0|
 0| 0| 0| 0| 0|
 0| 0| 0| 0| 1|
 0| 0| 0| 0| 0|
+------------+-------------+------------+-------------+------------------+
only showing top 5 rows

In [13]:
# know more abt data
# Descriptive statistics - use colStats() method
import pyspark.mllib.stat as st
import numpy as np

numeric_cols = ['MOTHER_AGE_YEARS','FATHER_COMBINED_AGE',
                'CIG_BEFORE','CIG_1_TRI','CIG_2_TRI','CIG_3_TRI',
                'MOTHER_HEIGHT_IN','MOTHER_PRE_WEIGHT',
                'MOTHER_DELIVERY_WEIGHT','MOTHER_WEIGHT_GAIN'
               ]

numeric_rdd = births_transformed\
                       .select(numeric_cols)\
                       .rdd \
                       .map(lambda row: [e for e in row])

mllib_stats = st.Statistics.colStats(numeric_rdd)

for col, m, v in zip(numeric_cols, 
                     mllib_stats.mean(), 
                     mllib_stats.variance()):
    print('{0}: \t{1:.2f} \t {2:.2f}'.format(col, m, np.sqrt(v)))


MOTHER_AGE_YEARS: 28.30 6.08
FATHER_COMBINED_AGE: 44.55 27.55
CIG_BEFORE: 1.43 5.18
CIG_1_TRI: 0.91 3.83
CIG_2_TRI: 0.70 3.31
CIG_3_TRI: 0.58 3.11
MOTHER_HEIGHT_IN: 65.12 6.45
MOTHER_PRE_WEIGHT: 214.50 210.21
MOTHER_DELIVERY_WEIGHT: 223.63 180.01
MOTHER_WEIGHT_GAIN: 30.74 26.23

In [14]:
# calculate the frequencies of values for the categorical variables.
categorical_cols = [e for e in births_transformed.columns 
                    if e not in numeric_cols]

categorical_rdd = births_transformed\
                       .select(categorical_cols)\
                       .rdd \
                       .map(lambda row: [e for e in row])
            
for i, col in enumerate(categorical_cols):
    agg = categorical_rdd \
        .groupBy(lambda row: row[i]) \
        .map(lambda row: (row[0], len(row[1])))
        
    print(col, sorted(agg.collect(), 
                      key=lambda el: el[1], 
                      reverse=True))


('INFANT_ALIVE_AT_REPORT', [(1, 23349), (0, 22080)])
('BIRTH_PLACE', [(u'1', 44558), (u'4', 327), (u'3', 224), (u'2', 136), (u'7', 91), (u'5', 74), (u'6', 11), (u'9', 8)])
('DIABETES_PRE', [(0, 44881), (1, 548)])
('DIABETES_GEST', [(0, 43451), (1, 1978)])
('HYP_TENS_PRE', [(0, 44348), (1, 1081)])
('HYP_TENS_GEST', [(0, 43302), (1, 2127)])
('PREV_BIRTH_PRETERM', [(0, 43088), (1, 2341)])

In [15]:
# Correlations between our features.


corrs = st.Statistics.corr(numeric_rdd)

for i, el in enumerate(corrs > 0.5):
    correlated = [
        (numeric_cols[j], corrs[i][j]) 
        for j, e in enumerate(el) 
        if e == 1.0 and j != i]
    
    if len(correlated) > 0:
        for e in correlated:
            print('{0}-to-{1}: {2:.2f}' \
                  .format(numeric_cols[i], e[0], e[1]))

CIG_BEFORE-to-CIG_1_TRI: 0.83
CIG_BEFORE-to-CIG_2_TRI: 0.72
CIG_BEFORE-to-CIG_3_TRI: 0.62
CIG_1_TRI-to-CIG_BEFORE: 0.83
CIG_1_TRI-to-CIG_2_TRI: 0.87
CIG_1_TRI-to-CIG_3_TRI: 0.76
CIG_2_TRI-to-CIG_BEFORE: 0.72
CIG_2_TRI-to-CIG_1_TRI: 0.87
CIG_2_TRI-to-CIG_3_TRI: 0.89
CIG_3_TRI-to-CIG_BEFORE: 0.62
CIG_3_TRI-to-CIG_1_TRI: 0.76
CIG_3_TRI-to-CIG_2_TRI: 0.89
MOTHER_PRE_WEIGHT-to-MOTHER_DELIVERY_WEIGHT: 0.54
MOTHER_PRE_WEIGHT-to-MOTHER_WEIGHT_GAIN: 0.65
MOTHER_DELIVERY_WEIGHT-to-MOTHER_PRE_WEIGHT: 0.54
MOTHER_DELIVERY_WEIGHT-to-MOTHER_WEIGHT_GAIN: 0.60
MOTHER_WEIGHT_GAIN-to-MOTHER_PRE_WEIGHT: 0.65
MOTHER_WEIGHT_GAIN-to-MOTHER_DELIVERY_WEIGHT: 0.60

In [16]:
# drop the highly correlated features
features_to_keep = [
    'INFANT_ALIVE_AT_REPORT', 
    'BIRTH_PLACE', 
    'MOTHER_AGE_YEARS', 
    'FATHER_COMBINED_AGE', 
    'CIG_1_TRI', 
    'MOTHER_HEIGHT_IN', 
    'MOTHER_PRE_WEIGHT', 
    'DIABETES_PRE', 
    'DIABETES_GEST', 
    'HYP_TENS_PRE', 
    'HYP_TENS_GEST', 
    'PREV_BIRTH_PRETERM'
]
births_transformed = births_transformed.select([e for e in features_to_keep])



In [17]:
# Run a Chi-square test to determine if there are significant differences for categorical variables.

import pyspark.mllib.linalg as ln

for cat in categorical_cols[1:]:
    agg = births_transformed \
        .groupby('INFANT_ALIVE_AT_REPORT') \
        .pivot(cat) \
        .count()    

    agg_rdd = agg \
        .rdd\
        .map(lambda row: (row[1:])) \
        .flatMap(lambda row: 
                 [0 if e == None else e for e in row]) \
        .collect()

    row_length = len(agg.collect()[0]) - 1
    agg = ln.Matrices.dense(row_length, 2, agg_rdd)
    
    test = st.Statistics.chiSqTest(agg)
    print(cat, round(test.pValue, 4))

('BIRTH_PLACE', 0.0)
('DIABETES_PRE', 0.0)
('DIABETES_GEST', 0.0)
('HYP_TENS_PRE', 0.0)
('HYP_TENS_GEST', 0.0)
('PREV_BIRTH_PRETERM', 0.0)

In [18]:
# Final dataset creatation
# use hashing to encode BIRTH_PLACE
import pyspark.mllib.feature as ft
import pyspark.mllib.regression as reg

hashing = ft.HashingTF(7)

births_hashed = births_transformed \
    .rdd \
    .map(lambda row: [
            list(hashing.transform(row[1]).toArray()) 
                if col == 'BIRTH_PLACE' 
                else row[i] 
            for i, col 
            in enumerate(features_to_keep)]) \
    .map(lambda row: [[e] if type(e) == int else e 
                      for e in row]) \
    .map(lambda row: [item for sublist in row 
                      for item in sublist]) \
    .map(lambda row: reg.LabeledPoint(
            row[0], 
            ln.Vectors.dense(row[1:]))
        )


In [19]:
births_transformed.show()


+----------------------+-----------+----------------+-------------------+---------+----------------+-----------------+------------+-------------+------------+-------------+------------------+
INFANT_ALIVE_AT_REPORT|BIRTH_PLACE|MOTHER_AGE_YEARS|FATHER_COMBINED_AGE|CIG_1_TRI|MOTHER_HEIGHT_IN|MOTHER_PRE_WEIGHT|DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|
+----------------------+-----------+----------------+-------------------+---------+----------------+-----------------+------------+-------------+------------+-------------+------------------+
 0| 1| 29| 99| 0| 99| 999| 0| 0| 0| 0| 0|
 0| 1| 22| 29| 0| 65| 180| 0| 0| 0| 0| 0|
 0| 1| 38| 40| 0| 63| 155| 0| 0| 0| 0| 0|
 0| 1| 39| 42| 0| 60| 128| 0| 0| 0| 0| 1|
 0| 1| 18| 99| 4| 61| 110| 0| 0| 0| 0| 0|
 0| 1| 32| 37| 0| 66| 150| 0| 0| 0| 0| 0|
 0| 1| 22| 25| 0| 68| 155| 0| 0| 0| 0| 0|
 0| 1| 25| 26| 0| 64| 136| 0| 0| 0| 0| 0|
 0| 1| 26| 32| 0| 64| 140| 0| 0| 0| 0| 0|
 0| 1| 39| 66| 0| 65| 140| 0| 0| 0| 0| 0|
 0| 1| 25| 22| 0| 62| 145| 0| 0| 0| 0| 0|
 0| 1| 33| 99| 0| 65| 145| 0| 0| 0| 0| 0|
 0| 1| 25| 99| 0| 64| 140| 0| 0| 0| 0| 0|
 0| 1| 29| 99| 0| 60| 115| 0| 0| 0| 0| 0|
 0| 1| 28| 29| 0| 66| 320| 0| 0| 1| 0| 1|
 0| 1| 23| 28| 0| 64| 120| 0| 0| 0| 0| 0|
 0| 1| 31| 41| 0| 59| 106| 0| 0| 0| 0| 1|
 0| 1| 27| 99| 0| 66| 213| 0| 0| 0| 0| 0|
 0| 1| 28| 27| 0| 66| 165| 0| 0| 0| 0| 1|
 0| 1| 34| 31| 0| 70| 130| 0| 0| 0| 0| 0|
+----------------------+-----------+----------------+-------------------+---------+----------------+-----------------+------------+-------------+------------+-------------+------------------+
only showing top 20 rows

In [20]:
# Split into training and testing

births_train, births_test = births_hashed.randomSplit([0.6, 0.4])


In [21]:
# Predicting infant survival
from pyspark.mllib.classification \
    import LogisticRegressionWithLBFGS

LR_Model = LogisticRegressionWithLBFGS \
    .train(births_train, iterations=10)

In [22]:
# use the model to predict the classes for our testing set.


LR_results = (
        births_test.map(lambda row: row.label) \
        .zip(LR_Model \
             .predict(births_test\
                      .map(lambda row: row.features)))
    ).map(lambda row: (row[0], row[1] * 1.0))

In [23]:
print(LR_results)

PythonRDD[8614] at RDD at PythonRDD.scala:50

In [24]:
import pyspark.mllib.evaluation as ev
LR_evaluation = ev.BinaryClassificationMetrics(LR_results)

print('Area under PR: {0:.2f}' \
      .format(LR_evaluation.areaUnderPR))
print('Area under ROC: {0:.2f}' \
      .format(LR_evaluation.areaUnderROC))
LR_evaluation.unpersist()

Area under PR: 0.84
Area under ROC: 0.63

In [25]:
# MLLib allows us to select the most predictable features using a Chi-Square selector.


selector = ft.ChiSqSelector(4).fit(births_train)

topFeatures_train = (
        births_train.map(lambda row: row.label) \
        .zip(selector \
             .transform(births_train \
                        .map(lambda row: row.features)))
    ).map(lambda row: reg.LabeledPoint(row[0], row[1]))

topFeatures_test = (
        births_test.map(lambda row: row.label) \
        .zip(selector \
             .transform(births_test \
                        .map(lambda row: row.features)))
    ).map(lambda row: reg.LabeledPoint(row[0], row[1]))

In [26]:
# now build the random forest model
from pyspark.mllib.tree import RandomForest

RF_model = RandomForest \
    .trainClassifier(data=topFeatures_train, 
                     numClasses=2, 
                     categoricalFeaturesInfo={}, 
                     numTrees=6,  
                     featureSubsetStrategy='all',
                     seed=666)

In [27]:
# check the accuracy with PR and ROC
RF_results = (
        topFeatures_test.map(lambda row: row.label) \
        .zip(RF_model \
             .predict(topFeatures_test \
                      .map(lambda row: row.features)))
    )

RF_evaluation = ev.BinaryClassificationMetrics(RF_results)

print('Area under PR: {0:.2f}' \
      .format(RF_evaluation.areaUnderPR))
print('Area under ROC: {0:.2f}' \
      .format(RF_evaluation.areaUnderROC))
RF_evaluation.unpersist()

Area under PR: 0.78
Area under ROC: 0.60

In [28]:
# now do the logistic regression with important features only
LR_Model_2 = LogisticRegressionWithLBFGS \
    .train(topFeatures_train, iterations=10)

LR_results_2 = (
        topFeatures_test.map(lambda row: row.label) \
        .zip(LR_Model_2 \
             .predict(topFeatures_test \
                      .map(lambda row: row.features)))
    ).map(lambda row: (row[0], row[1] * 1.0))

LR_evaluation_2 = ev.BinaryClassificationMetrics(LR_results_2)

print('Area under PR: {0:.2f}' \
      .format(LR_evaluation_2.areaUnderPR))
print('Area under ROC: {0:.2f}' \
      .format(LR_evaluation_2.areaUnderROC))
LR_evaluation_2.unpersist()

Area under PR: 0.87
Area under ROC: 0.62